In [1]:
# Imports
import os
import requests
import pandas as pd
import datetime
from twilio.rest import Client

In [2]:
# GLOBAL API Setup
ALPHA_API_KEY = os.environ.get('ALPHA_API_KEY')
NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
TWILIO_ACCOUNT_SID = os.environ.get('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.environ.get('TWILIO_AUTH_TOKEN')
FROM_WPP = os.environ.get('FROM_WPP')
TO_WPP = os.environ.get('TO_WPP')
ALPHA_URL = 'https://www.alphavantage.co/query?'
NEWS_URL = 'https://newsapi.org/v2/everything'


In [3]:

# Stock variation calculation
def get_stock_variation(stock_symbol: str) -> bool:
    """Fetch stock variation percentage from the Alpha Vange API and check if it exceeds 5%."""
    alpha_params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': stock_symbol.upper(),
        'apikey': ALPHA_API_KEY
    }
    
    
    
    response = requests.get(ALPHA_URL, params=alpha_params)
    response.raise_for_status()
    raw_data = response.json()
    print(raw_data)
    data = raw_data["Time Series (Daily)"]
    #print(data)
    
    dates = sorted(data.keys(), reverse=True) # Get sorted available dates from API
    print(dates)
    
    try:
        yesterday_date = dates[0]
        two_days_ago_date = dates[1]
    except IndexError:
        print("Not enough data to calculate stock variation")
        return False
    
    # Fetch close prices
    yesterday_close = float(data[yesterday_date]["4. close"])
    two_days_close = float(data[two_days_ago_date]["4. close"])
    
    percentage_difference = (yesterday_close - two_days_close) / two_days_close * 100
    print(f'Stock price variation:{percentage_difference}')
    
    if abs(percentage_difference) >= 5:
        print("The variation is more than 5%")
        return True
    else:
        print("The variation is within 5%")
        return False

In [4]:
# Fetch news articles
def get_news(query) -> list:
    """Retrieve top 3 news articles based on the query."""
    
    yesterday_date = datetime.datetime.now() - datetime.timedelta(days=1)
    news_params = {
        'apiKey': os.environ.get('NEWS_API_KEY'),
        'q': query,
        'from': yesterday_date,
        'language': 'en',
        'sortBy':'relevancy'
    }
    
    response = requests.get(NEWS_URL, news_params)
    response.raise_for_status()
    data = response.json() # o: python dict
    # print(data)
    # print(type(data))

    return data['articles'][0:3]

In [5]:
# Send wpp message
def set_wpp_message(message):
    """Send a WhatsApp message using Twilio."""
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    msn = client.messages.create( 
        body=message,
        from_=FROM_WPP,
        to=TO_WPP,
        )
    return msn.status

In [6]:
# TEST
stock_price_variation = get_stock_variation("AAPL")
print(stock_price_variation)
news = get_news("apple")
print(news)
message= "testing MESSAGE. From: test."
msn = set_wpp_message(message)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'AAPL', '3. Last Refreshed': '2025-01-27', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-01-27': {'1. open': '224.0200', '2. high': '232.1500', '3. low': '223.9800', '4. close': '229.8600', '5. volume': '94863418'}, '2025-01-24': {'1. open': '224.7800', '2. high': '225.6300', '3. low': '221.4100', '4. close': '222.7800', '5. volume': '54697907'}, '2025-01-23': {'1. open': '224.7400', '2. high': '227.0300', '3. low': '222.3000', '4. close': '223.6600', '5. volume': '60234760'}, '2025-01-22': {'1. open': '219.7900', '2. high': '224.1200', '3. low': '219.7900', '4. close': '223.8300', '5. volume': '64126500'}, '2025-01-21': {'1. open': '224.0000', '2. high': '224.4200', '3. low': '219.3800', '4. close': '222.6400', '5. volume': '98070429'}, '2025-01-17': {'1. open': '232.1150', '2. high': '232.2900', '3. low': '228.4800', '4. close': '229.9800', '5

In [10]:
def main():
    stock_price_variation = get_stock_variation("NVDA")
    topic= "Nvidia"
    news = get_news(topic)
    if stock_price_variation:
        print("Stock variation detected. Fetching news articles...")
        print(news)
        #for article in news:
        #    message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
        #    set_wpp_message(message)
        #    print(f"Message sent: {message}")
    else:
        print("No stock variation detected")
        #print(news)
        #message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
        #set_wpp_message(message)
        #print(f"Message sent: {message}")
        
        

In [11]:
main()

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'NVDA', '3. Last Refreshed': '2025-01-27', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-01-27': {'1. open': '124.8000', '2. high': '128.4000', '3. low': '116.7000', '4. close': '118.4200', '5. volume': '818830895'}, '2025-01-24': {'1. open': '148.3700', '2. high': '148.9700', '3. low': '141.8800', '4. close': '142.6200', '5. volume': '234657635'}, '2025-01-23': {'1. open': '145.0500', '2. high': '147.2300', '3. low': '143.7200', '4. close': '147.2200', '5. volume': '155915549'}, '2025-01-22': {'1. open': '144.6600', '2. high': '147.7900', '3. low': '143.6700', '4. close': '147.0700', '5. volume': '237651395'}, '2025-01-21': {'1. open': '139.1600', '2. high': '141.8300', '3. low': '137.0900', '4. close': '140.8300', '5. volume': '197748985'}, '2025-01-17': {'1. open': '136.6900', '2. high': '138.5000', '3. low': '135.4649', '4. close': '137.7100